# Xây dựng Agent với Function calling

## Agent là gì?

Agent có thể hiểu là một chương trình AI có thể giải quyết những tác vụ phức tạp sử dụng những công cụ mà nó được tích hợp.

## Các thành phần chính:
- Thành phần suy luận: agent sử dụng LLM để suy luận
- Agent được trang bị các công cụ (tools)

## Agent với Function calling
Azure Open AI hỗ trợ tính năng Function calling, giúp xây dựng agent một cách dễ dàng.
Chúng ta cần định nghĩa các công cụ (tools) mà agent có thể sử dụng. 

## Setup API clients

In [25]:
import os
import dotenv
import openai

dotenv.load_dotenv()

# Initialize Azure OpenAI client
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

client = openai.AzureOpenAI(
    api_version="2024-05-01-preview",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com")

## Định nghĩa các công cụ

Ở đây, chúng ta định nghĩa một công cụ có tên "get_current_time" có nhiệm vụ lấy thông tin thời gian hiện tại của hệ thống.
Chúng ta cần định nghĩa các thành phần sau:
- name
- description: Mô tả chức năng của function
- parameters: Mô tả các tham số mà function nhận

In [ ]:
# Define the function for the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time in a given timezone",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone": {
                        "type": "string",
                        "description": "The timezone name of the city, e.g Asia/Tokyo, America/Los_Angeles",
                    },
                },
                "required": ["location"],
            },
        }
    }
]

In [27]:
import os
import json
from datetime import datetime
from typing import List
from zoneinfo import ZoneInfo

# Run the conversation and print the result
messages = []

def get_current_time(timezone):
    """Get the current time for a given timezone"""
    print(f"get_current_time called with timezone: {timezone}")  
    if timezone is None:
        return "No timezone provided"
    
    current_time = datetime.now(ZoneInfo(timezone)).strftime("%I:%M %p")
    return json.dumps({        
        "current_time": current_time
    })
    
   

def run_conversation(question:str, messages:List[dict]=[]):      
    messages.append({"role": "user", "content": question})    

    # First API call: Ask the model to use the function
    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    # Process the model's response
    response_message = response.choices[0].message
    messages.append(response_message)

    print("Model's response:")  
    print(response_message)  

    # Handle function calls
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            if tool_call.function.name == "get_current_time":
                function_args = json.loads(tool_call.function.arguments)
                print(f"Function arguments: {function_args}")  
                time_response = get_current_time(
                    timezone=function_args.get("timezone")
                )
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "get_current_time",
                    "content": time_response,
                })
    else:
        print("No tool calls were made by the model.")  

    # Second API call: Get the final response from the model
    final_response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=messages,
    )

    return final_response.choices[0].message.content


In [29]:

print(run_conversation("What's the current time in Hanoi", messages=messages))

Model's response:
ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2UiHXwJhtnt0s9vFevITB0W6', function=Function(arguments='{"timezone":"Asia/Ho_Chi_Minh"}', name='get_current_time'), type='function')])
Function arguments: {'timezone': 'Asia/Ho_Chi_Minh'}
get_current_time called with timezone: Asia/Ho_Chi_Minh
The current time in Hanoi is 1:13 PM.
